# 形態素解析の実装
1. クリーニング
2. 正規化
3. 形態素解析
4. 基本形への変換
5. stop wordの除去
6. 単語の数値化

aws上で<br>
source activate pytorch_p36<br>
pip install janome

## Library

In [1]:
from janome.tokenizer import Tokenizer
j_t = Tokenizer()
text = '機械学習が好きです'
# 英語を入れるとどうなるのか
text2 = 'splatoon2とスマブラとPocket Monstersとぷよぷよが好きです'
# 古文だとどうなるか
text3 ='今は昔、竹取の翁といふものありけり。野山にまじりて竹を取りつつ、よろづのことに使ひけり。' 

In [2]:
for token in j_t.tokenize(text):
    print(token)

機械	名詞,一般,*,*,*,*,機械,キカイ,キカイ
学習	名詞,サ変接続,*,*,*,*,学習,ガクシュウ,ガクシュー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス


In [3]:
for token in j_t.tokenize(text2):
    print(token)
    
# splatoonと2に分かれる
# スマとブラに分かれる
# ぷよぷよが　が一つの名詞となる

splatoon	名詞,一般,*,*,*,*,splatoon,*,*
2	名詞,数,*,*,*,*,2,*,*
と	助詞,並立助詞,*,*,*,*,と,ト,ト
スマ	名詞,固有名詞,一般,*,*,*,スマ,スマ,スマ
ブラ	名詞,一般,*,*,*,*,ブラ,ブラ,ブラ
と	助詞,並立助詞,*,*,*,*,と,ト,ト
Pocket	名詞,固有名詞,組織,*,*,*,Pocket,*,*
 	記号,空白,*,*,*,*, ,*,*
Monsters	名詞,固有名詞,組織,*,*,*,Monsters,*,*
と	助詞,並立助詞,*,*,*,*,と,ト,ト
ぷよぷよが	名詞,一般,*,*,*,*,ぷよぷよが,*,*
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス


In [4]:
for token in j_t.tokenize(text3):
    print(token)

# よとづのことにがばらばらになる
# けり　の　文語とは？　→　文章だけに用いる言語

今	名詞,副詞可能,*,*,*,*,今,イマ,イマ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
昔	名詞,副詞可能,*,*,*,*,昔,ムカシ,ムカシ
、	記号,読点,*,*,*,*,、,、,、
竹	名詞,一般,*,*,*,*,竹,タケ,タケ
取	動詞,自立,*,*,五段・ラ行,体言接続特殊２,取る,ト,ト
の	助詞,連体化,*,*,*,*,の,ノ,ノ
翁	名詞,固有名詞,人名,姓,*,*,翁,オウ,オウ
といふ	助詞,格助詞,連語,*,*,*,といふ,トイフ,トユウ
もの	名詞,非自立,一般,*,*,*,もの,モノ,モノ
あり	助動詞,*,*,*,五段・ラ行アル,連用形,ある,アリ,アリ
けり	助動詞,*,*,*,文語・ケリ,基本形,けり,ケリ,ケリ
。	記号,句点,*,*,*,*,。,。,。
野山	名詞,一般,*,*,*,*,野山,ノヤマ,ノヤマ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
まじり	動詞,自立,*,*,五段・ラ行,連用形,まじる,マジリ,マジリ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
竹	名詞,一般,*,*,*,*,竹,タケ,タケ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
取り	動詞,自立,*,*,五段・ラ行,連用形,取る,トリ,トリ
つつ	助詞,接続助詞,*,*,*,*,つつ,ツツ,ツツ
、	記号,読点,*,*,*,*,、,、,、
よろ	動詞,自立,*,*,五段・ラ行,未然ウ接続,よる,ヨロ,ヨロ
づのことに	名詞,一般,*,*,*,*,づのことに,*,*
使	名詞,接尾,一般,*,*,*,使,シ,シ
ひ	動詞,自立,*,*,一段,連用形,ひる,ヒ,ヒ
けり	助動詞,*,*,*,文語・ケリ,基本形,けり,ケリ,ケリ
。	記号,句点,*,*,*,*,。,。,。


## tokenizer_janomeの実装
janomeの処理を機械学習実装時に使用しやすいように関数化したもの<br>
単語の瀕死が必要ない場合は j_t.tokenizeの引数に wakati=True

In [5]:
def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]

text = '機械学習が好きです'
tokenizer_janome(text)

['機械', '学習', 'が', '好き', 'です']

In [6]:
# wakati=Falseだと
def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=False)]

text = '機械学習が好きです'
tokenizer_janome(text)

In [7]:
# 追加辞書を使わない場合
import MeCab
m_t = MeCab.Tagger('-Ochasen')
text = '機械学習が好きです'
print(m_t.parse(text))

機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
EOS



In [8]:
# 追加辞書　neologdを使った場合
m_t = MeCab.Tagger('-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
text = '機械学習が好きです'
print(m_t.parse(text))

機械学習	キカイガクシュウ	機械学習	名詞-固有名詞-一般		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
EOS



In [9]:
print(m_t.parse(text2))
# splatoon2とスマブラが一つの単語として認識された！

splatoon2	スプラトゥーンツー	Splatoon2	名詞-固有名詞-一般		
と	ト	と	助詞-並立助詞		
スマブラ	スマブラ	スマブラ	名詞-固有名詞-一般		
と	ト	と	助詞-並立助詞		
Pocket	ポケット	Pocket	名詞-固有名詞-一般		
Monsters	モンスターズ	MONSTERS	名詞-固有名詞-人名-一般		
と	ト	と	助詞-並立助詞		
ぷよぷよ	プヨプヨ	ぷよぷよ	名詞-固有名詞-一般		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
EOS



# torchtextを用いたDataset, DataLoaderの実装

## 前処理と単語分割の関数を実装

In [10]:
# 単語分割にはJanomeを使用
from janome.tokenizer import Tokenizer
j_t = Tokenizer()

def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]

In [11]:
# 前処理として正規化する関数
import re

def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(' ', '', text)
    text = re.sub('　', '', text)
    
    # 数字文字の一律「０」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # rは''の外
    
    return text

In [12]:
#  前処理と単語分割を合わせた関数を定義する(上の２つの関数)

def tokenizer_with_preprocessing(text):
        text = preprocessing_text(text)  # 正規化
        ret = tokenizer_janome(text)       # 単語分割 
        
        return ret
    
# 動作確認
text = '昨日はとても暑く、気温が36度もあった。'
print(tokenizer_with_preprocessing(text))

['昨日', 'は', 'とても', '暑く', '、', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


## 文章データの読み込み

In [13]:
import torchtext

# tsv, csvを読み込んだときに行う処理を定義
# 文章とラベルの療法に用意する

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, 
                           use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 引数の意味は次の通り
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリー（単語集：後で解説）に追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を先頭に用意するかどうか
# fix_length：全部の文章を指定した長さと同じになるように、paddingまたは切り捨てします

In [23]:
# data.TabularDataset  Datasetに変換するクラス
# フォルダ「data」から各tsvファイルを読み込みDatastにする
# 1行がTEXTとLABELで区切られていることをfieldsで指示する

train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv', 
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])
# FieldとそのFieldに対する処理（上で定義したもの）

# 動作確認
print('訓練データの数', len(train_ds))
print('１つ目の訓練データ', vars(train_ds[0]))
print('2つ目の訓練データ', vars(train_ds[1]))

訓練データの数 4
１つ目の訓練データ {'Text': ['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。'], 'Label': '0'}
2つ目の訓練データ {'Text': ['機械', '学習', 'が', '好き', 'です', '。'], 'Label': '1'}


## 単語の数値化

In [17]:
# ボキャブラリーの作成
# 訓練データの単語からmin_freq以上の頻度の単語を使用してボキャブラリーを構築
TEXT.build_vocab(train_ds, min_freq=1)

# 訓練データ内の単語と頻度を出力
TEXT.vocab.freqs


Counter({'王': 1,
         'と': 5,
         '王子': 1,
         '女王': 1,
         '姫': 1,
         '男性': 1,
         '女性': 1,
         'が': 3,
         'い': 1,
         'まし': 1,
         'た': 1,
         '。': 4,
         '機械': 1,
         '学習': 1,
         '好き': 1,
         'です': 1,
         '本章': 2,
         'から': 1,
         '自然': 1,
         '言語': 1,
         '処理': 1,
         'に': 1,
         '取り組み': 1,
         'ます': 2,
         'で': 1,
         'は': 1,
         '商品': 1,
         'レビュー': 1,
         'の': 4,
         '短い': 1,
         '文章': 4,
         'に対して': 1,
         '、': 3,
         'その': 1,
         'ネガティブ': 1,
         'な': 4,
         '評価': 2,
         'を': 3,
         'し': 3,
         'て': 2,
         'いる': 2,
         'か': 2,
         'ポジティブ': 1,
         '0': 1,
         '値': 1,
         'クラス': 1,
         '分類': 2,
         'する': 1,
         'モデル': 1,
         '構築': 1})

In [18]:
# ボキャブラリーの単語をidに変換した結果を出力
# min_freqより小さい場合は<unk>になる
TEXT.vocab.stoi  # string to identifiers 文字列をidへ

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f759eb77c88>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
     

## DataLoaderの作成

In [22]:
# torchtextの文脈では単純にiteraterと呼ばれている
train_dl = torchtext.data.Iterator(train_ds, batch_size=2, train=True)
val_dl = torchtext.data.Iterator(val_ds, batch_size=2, train=False, sort=False)
test_dl = torchtext.data.Iterator(test_ds, batch_size=2, train=False, sort=False)

# 動作確認　検証データセットで確認
# 一番目の用を取り出す
batch = next(iter(val_dl))
# textをidで表す
print(batch.Text)
print(batch.Label)

(tensor([[46,  2, 47,  2, 40,  2, 42,  2, 48,  2, 39,  8, 19, 29, 23,  3,  1,  1,
          1,  1,  1,  1,  1,  1,  1],
        [45, 43,  8, 41, 25,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1]]), tensor([16,  6]))
tensor([0, 1])
